In [1]:
from pyspark.sql import Row

In [2]:
data = [('Alice', 1), ('Bob', 2), ('Bill', 4)]
df = sqlContext.createDataFrame(data, ['name', 'age'])
fil = df.filter(df.age > 3).collect()
print(fil)

# If the Spark job doesn't work properly this will raise an AssertionError
assert fil == [Row(u'Bill', 4)]


[Row(name='Bill', age=4)]


In [4]:
type(sqlContext)

pyspark.sql.context.SQLContext

In [5]:
dir(sqlContext)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_inferSchema',
 '_instantiatedContext',
 '_jsc',
 '_jsqlContext',
 '_jvm',
 '_sc',
 '_ssql_ctx',
 'cacheTable',
 'clearCache',
 'createDataFrame',
 'createExternalTable',
 'dropTempTable',
 'getConf',
 'getOrCreate',
 'newSession',
 'range',
 'read',
 'readStream',
 'registerDataFrameAsTable',
 'registerFunction',
 'registerJavaFunction',
 'setConf',
 'sparkSession',
 'sql',
 'streams',
 'table',
 'tableNames',
 'tables',
 'udf',
 'uncacheTable']

In [6]:
help(sqlContext)

Help on SQLContext in module pyspark.sql.context object:

class SQLContext(builtins.object)
 |  The entry point for working with structured data (rows and columns) in Spark, in Spark 1.x.
 |  
 |  As of Spark 2.0, this is replaced by :class:`SparkSession`. However, we are keeping the class
 |  here for backward compatibility.
 |  
 |  A SQLContext can be used create :class:`DataFrame`, register :class:`DataFrame` as
 |  tables, execute SQL over tables, cache tables, and read parquet files.
 |  
 |  :param sparkContext: The :class:`SparkContext` backing this SQLContext.
 |  :param sparkSession: The :class:`SparkSession` around which this SQLContext wraps.
 |  :param jsqlContext: An optional JVM Scala SQLContext. If set, we do not instantiate a new
 |      SQLContext in the JVM, instead we make all calls to this object.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, sparkContext, sparkSession=None, jsqlContext=None)
 |      Creates a new SQLContext.
 |      
 |      >>> from date

In [7]:
sc.version

'2.2.0'

In [8]:
help(map)

Help on class map in module builtins:

class map(object)
 |  map(func, *iterables) --> map object
 |  
 |  Make an iterator that computes the function using arguments from
 |  each of the iterables.  Stops when the shortest iterable is exhausted.
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.



In [10]:
from faker import Factory
fake = Factory.create()
fake.seed(4321)

In [11]:
# Each entry consists of last_name, first_name, ssn, job, and age (at least 1)
from pyspark.sql import Row
def fake_entry():
  name = fake.name().split()
  return (name[1], name[0], fake.ssn(), fake.job(), abs(2016 - fake.date_time().year) + 1)

In [14]:
# Create a helper function to call a function repeatedly
def repeat(times, func, *args, **kwargs):
    for _ in range(times):
        yield func(*args, **kwargs)

In [15]:
data = list(repeat(10000, fake_entry))

In [16]:
data[0]

('Brown', 'Jason', '182-83-5988', 'Clothing/textile technologist', 2)

In [17]:
len(data)

10000

In [18]:
help(sqlContext.createDataFrame)

Help on method createDataFrame in module pyspark.sql.context:

createDataFrame(data, schema=None, samplingRatio=None, verifySchema=True) method of pyspark.sql.context.SQLContext instance
    Creates a :class:`DataFrame` from an :class:`RDD`, a list or a :class:`pandas.DataFrame`.
    
    When ``schema`` is a list of column names, the type of each column
    will be inferred from ``data``.
    
    When ``schema`` is ``None``, it will try to infer the schema (column names and types)
    from ``data``, which should be an RDD of :class:`Row`,
    or :class:`namedtuple`, or :class:`dict`.
    
    When ``schema`` is :class:`pyspark.sql.types.DataType` or a datatype string it must match
    the real data, or an exception will be thrown at runtime. If the given schema is not
    :class:`pyspark.sql.types.StructType`, it will be wrapped into a
    :class:`pyspark.sql.types.StructType` as its only field, and the field name will be "value",
    each record will also be wrapped into a tuple, wh

In [19]:
dataDF = sqlContext.createDataFrame(data, ('last_name', 'first_name', 'ssn', 'occupation', 'age'))

In [20]:
print('type of dataDF: {0}'.format(type(dataDF)))

type of dataDF: <class 'pyspark.sql.dataframe.DataFrame'>


In [21]:
dataDF.printSchema()

root
 |-- last_name: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- ssn: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- age: long (nullable = true)



In [22]:
sqlContext.registerDataFrameAsTable(dataDF, 'dataframe')

In [23]:
help(dataDF)

Help on DataFrame in module pyspark.sql.dataframe object:

class DataFrame(builtins.object)
 |  A distributed collection of data grouped into named columns.
 |  
 |  A :class:`DataFrame` is equivalent to a relational table in Spark SQL,
 |  and can be created using various functions in :class:`SQLContext`::
 |  
 |      people = sqlContext.read.parquet("...")
 |  
 |  Once created, it can be manipulated using the various domain-specific-language
 |  (DSL) functions defined in: :class:`DataFrame`, :class:`Column`.
 |  
 |  To select a column from the data frame, use the apply method::
 |  
 |      ageCol = people.age
 |  
 |  A more concrete example::
 |  
 |      # To create DataFrame using SQLContext
 |      people = sqlContext.read.parquet("...")
 |      department = sqlContext.read.parquet("...")
 |  
 |      people.filter(people.age > 30).join(department, people.deptId == department.id) \
 |        .groupBy(department.name, "gender").agg({"salary": "avg", "age": "max"})
 |  
 |  ..

In [24]:
dataDF.rdd.getNumPartitions()

4

In [25]:
newDF = dataDF.distinct().select('*')
newDF.explain(True)

== Parsed Logical Plan ==
'Project [*]
+- Deduplicate [first_name#7, age#10L, last_name#6, occupation#9, ssn#8], false
   +- LogicalRDD [last_name#6, first_name#7, ssn#8, occupation#9, age#10L]

== Analyzed Logical Plan ==
last_name: string, first_name: string, ssn: string, occupation: string, age: bigint
Project [last_name#6, first_name#7, ssn#8, occupation#9, age#10L]
+- Deduplicate [first_name#7, age#10L, last_name#6, occupation#9, ssn#8], false
   +- LogicalRDD [last_name#6, first_name#7, ssn#8, occupation#9, age#10L]

== Optimized Logical Plan ==
Aggregate [first_name#7, age#10L, last_name#6, occupation#9, ssn#8], [last_name#6, first_name#7, ssn#8, occupation#9, age#10L]
+- LogicalRDD [last_name#6, first_name#7, ssn#8, occupation#9, age#10L]

== Physical Plan ==
*HashAggregate(keys=[first_name#7, age#10L, last_name#6, occupation#9, ssn#8], functions=[], output=[last_name#6, first_name#7, ssn#8, occupation#9, age#10L])
+- Exchange hashpartitioning(first_name#7, age#10L, last_name#6

In [26]:
# Transform dataDF through a select transformation and rename the newly created '(age -1)' column to 'age'
# Because select is a transformation and Spark uses lazy evaluation, no jobs, stages,
# or tasks will be launched when we run this code.
subDF = dataDF.select('last_name', 'first_name', 'ssn', 'occupation', (dataDF.age - 1).alias('age'))

In [27]:
subDF.explain(True)

== Parsed Logical Plan ==
'Project [unresolvedalias('last_name, None), unresolvedalias('first_name, None), unresolvedalias('ssn, None), unresolvedalias('occupation, None), (age#10L - 1) AS age#25]
+- LogicalRDD [last_name#6, first_name#7, ssn#8, occupation#9, age#10L]

== Analyzed Logical Plan ==
last_name: string, first_name: string, ssn: string, occupation: string, age: bigint
Project [last_name#6, first_name#7, ssn#8, occupation#9, (age#10L - cast(1 as bigint)) AS age#25L]
+- LogicalRDD [last_name#6, first_name#7, ssn#8, occupation#9, age#10L]

== Optimized Logical Plan ==
Project [last_name#6, first_name#7, ssn#8, occupation#9, (age#10L - 1) AS age#25L]
+- LogicalRDD [last_name#6, first_name#7, ssn#8, occupation#9, age#10L]

== Physical Plan ==
*Project [last_name#6, first_name#7, ssn#8, occupation#9, (age#10L - 1) AS age#25L]
+- Scan ExistingRDD[last_name#6,first_name#7,ssn#8,occupation#9,age#10L]


In [29]:
# Let's collect the data
results = subDF.collect()
print(results)

[Row(last_name='Brown', first_name='Jason', ssn='182-83-5988', occupation='Clothing/textile technologist', age=1), Row(last_name='Brown', first_name='Cody', ssn='298-53-9877', occupation='Exhibition designer', age=19), Row(last_name='Griffin', first_name='Sandra', ssn='175-58-0111', occupation='Clothing/textile technologist', age=10), Row(last_name='Wyatt', first_name='David', ssn='270-76-3455', occupation='Surveyor, mining', age=24), Row(last_name='George', first_name='Daniel', ssn='200-38-3837', occupation='Maintenance engineer', age=18), Row(last_name='Rogers', first_name='Barry', ssn='634-25-3185', occupation='Local government officer', age=30), Row(last_name='Foster', first_name='Morgan', ssn='464-88-6116', occupation='Presenter, broadcasting', age=39), Row(last_name='Hansen', first_name='John', ssn='773-12-5058', occupation='Armed forces logistics/support/administrative officer', age=42), Row(last_name='Rice', first_name='Derek', ssn='054-51-9007', occupation='Pharmacist, hospita

In [30]:
subDF.show()

+---------+----------+-----------+--------------------+---+
|last_name|first_name|        ssn|          occupation|age|
+---------+----------+-----------+--------------------+---+
|    Brown|     Jason|182-83-5988|Clothing/textile ...|  1|
|    Brown|      Cody|298-53-9877| Exhibition designer| 19|
|  Griffin|    Sandra|175-58-0111|Clothing/textile ...| 10|
|    Wyatt|     David|270-76-3455|    Surveyor, mining| 24|
|   George|    Daniel|200-38-3837|Maintenance engineer| 18|
|   Rogers|     Barry|634-25-3185|Local government ...| 30|
|   Foster|    Morgan|464-88-6116|Presenter, broadc...| 39|
|   Hansen|      John|773-12-5058|Armed forces logi...| 42|
|     Rice|     Derek|054-51-9007|Pharmacist, hospital| 23|
|   Little|    Cheryl|725-70-4549|Brewing technologist| 41|
|   Peters|      Leah|826-58-6908|Television produc...|  8|
|    Simon|     Logan|062-37-6157|Health service ma...| 31|
|  Bennett|   Krystal|308-43-0932|Emergency plannin...| 24|
|    Burke|     Kelly|299-52-4282|Aerona

In [32]:
subDF.show(n=30, truncate=True)

+---------+----------+-----------+--------------------+---+
|last_name|first_name|        ssn|          occupation|age|
+---------+----------+-----------+--------------------+---+
|    Brown|     Jason|182-83-5988|Clothing/textile ...|  1|
|    Brown|      Cody|298-53-9877| Exhibition designer| 19|
|  Griffin|    Sandra|175-58-0111|Clothing/textile ...| 10|
|    Wyatt|     David|270-76-3455|    Surveyor, mining| 24|
|   George|    Daniel|200-38-3837|Maintenance engineer| 18|
|   Rogers|     Barry|634-25-3185|Local government ...| 30|
|   Foster|    Morgan|464-88-6116|Presenter, broadc...| 39|
|   Hansen|      John|773-12-5058|Armed forces logi...| 42|
|     Rice|     Derek|054-51-9007|Pharmacist, hospital| 23|
|   Little|    Cheryl|725-70-4549|Brewing technologist| 41|
|   Peters|      Leah|826-58-6908|Television produc...|  8|
|    Simon|     Logan|062-37-6157|Health service ma...| 31|
|  Bennett|   Krystal|308-43-0932|Emergency plannin...| 24|
|    Burke|     Kelly|299-52-4282|Aerona

In [33]:
display(subDF)

DataFrame[last_name: string, first_name: string, ssn: string, occupation: string, age: bigint]

In [34]:
print(dataDF.count())
print(subDF.count())

10000
10000


In [35]:
filteredDF = subDF.filter(subDF.age < 10)
filteredDF.show(truncate=False)
filteredDF.count()

+----------+-----------+-----------+------------------------------------+---+
|last_name |first_name |ssn        |occupation                          |age|
+----------+-----------+-----------+------------------------------------+---+
|Brown     |Jason      |182-83-5988|Clothing/textile technologist       |1  |
|Peters    |Leah       |826-58-6908|Television production assistant     |8  |
|Burke     |Kelly      |299-52-4282|Aeronautical engineer               |4  |
|Robinson  |Paige      |212-42-9151|Clinical psychologist               |7  |
|Blair     |Emily      |528-14-5681|Chartered certified accountant      |2  |
|Patterson |Bobby      |063-74-9852|International aid/development worker|0  |
|Myers     |Samuel     |698-12-1613|Geologist, wellsite                 |1  |
|Fisher    |Steven     |898-49-3926|Advertising account planner         |2  |
|Hill      |Tanya      |644-05-1087|Research officer, government        |0  |
|Singleton |Brittney   |296-98-1289|Conservation officer, nature

2238

In [38]:
from pyspark.sql.types import BooleanType
from pyspark.sql.functions import udf
less_ten = udf(lambda s: s < 10, BooleanType())
lambdaDF = subDF.filter(less_ten(subDF.age))
lambdaDF.show()
lambdaDF.count()

+----------+-----------+-----------+--------------------+---+
| last_name| first_name|        ssn|          occupation|age|
+----------+-----------+-----------+--------------------+---+
|     Brown|      Jason|182-83-5988|Clothing/textile ...|  1|
|    Peters|       Leah|826-58-6908|Television produc...|  8|
|     Burke|      Kelly|299-52-4282|Aeronautical engi...|  4|
|  Robinson|      Paige|212-42-9151|Clinical psycholo...|  7|
|     Blair|      Emily|528-14-5681|Chartered certifi...|  2|
| Patterson|      Bobby|063-74-9852|International aid...|  0|
|     Myers|     Samuel|698-12-1613| Geologist, wellsite|  1|
|    Fisher|     Steven|898-49-3926|Advertising accou...|  2|
|      Hill|      Tanya|644-05-1087|Research officer,...|  0|
| Singleton|   Brittney|296-98-1289|Conservation offi...|  6|
|     Smith|    Brandon|780-25-1953|      Futures trader|  6|
|      King|    Matthew|263-65-2624|Learning disabili...|  7|
|   Navarro|    William|200-09-7607|Television produc...|  5|
|  Willi

2238

In [39]:
# Let's collect the even values less than 10
even = udf(lambda s: s % 2 == 0, BooleanType())
evenDF = lambdaDF.filter(even(lambdaDF.age))
evenDF.show()
evenDF.count()

+----------+----------+-----------+--------------------+---+
| last_name|first_name|        ssn|          occupation|age|
+----------+----------+-----------+--------------------+---+
|    Peters|      Leah|826-58-6908|Television produc...|  8|
|     Burke|     Kelly|299-52-4282|Aeronautical engi...|  4|
|     Blair|     Emily|528-14-5681|Chartered certifi...|  2|
| Patterson|     Bobby|063-74-9852|International aid...|  0|
|    Fisher|    Steven|898-49-3926|Advertising accou...|  2|
|      Hill|     Tanya|644-05-1087|Research officer,...|  0|
| Singleton|  Brittney|296-98-1289|Conservation offi...|  6|
|     Smith|   Brandon|780-25-1953|      Futures trader|  6|
|Richardson|     Randy|608-49-4728|   Company secretary|  4|
|     Johns|  Jennifer|516-20-7686|Engineer, civil (...|  4|
|     Brown|    Sheila|741-62-8267|Private music tea...|  8|
|   Morales|    Kendra|773-23-3100|Multimedia progra...|  0|
|      Long|      Alex|679-33-4028|       Stage manager|  2|
|     Mills|      Gary|2

1046

In [40]:
print("first: {0}\n".format(filteredDF.first()))

print("Four of them: {0}\n".format(filteredDF.take(4)))

first: Row(last_name='Brown', first_name='Jason', ssn='182-83-5988', occupation='Clothing/textile technologist', age=1)

Four of them: [Row(last_name='Brown', first_name='Jason', ssn='182-83-5988', occupation='Clothing/textile technologist', age=1), Row(last_name='Peters', first_name='Leah', ssn='826-58-6908', occupation='Television production assistant', age=8), Row(last_name='Burke', first_name='Kelly', ssn='299-52-4282', occupation='Aeronautical engineer', age=4), Row(last_name='Robinson', first_name='Paige', ssn='212-42-9151', occupation='Clinical psychologist', age=7)]



In [41]:
display(filteredDF.take(4))

[Row(last_name='Brown', first_name='Jason', ssn='182-83-5988', occupation='Clothing/textile technologist', age=1),
 Row(last_name='Peters', first_name='Leah', ssn='826-58-6908', occupation='Television production assistant', age=8),
 Row(last_name='Burke', first_name='Kelly', ssn='299-52-4282', occupation='Aeronautical engineer', age=4),
 Row(last_name='Robinson', first_name='Paige', ssn='212-42-9151', occupation='Clinical psychologist', age=7)]

In [42]:
display(dataDF.orderBy(dataDF.age.desc()).take(5))

[Row(last_name='Smith', first_name='Edward', ssn='051-21-6125', occupation='Mental health nurse', age=47),
 Row(last_name='Long', first_name='William', ssn='356-32-6651', occupation='Conservation officer, nature', age=47),
 Row(last_name='Miller', first_name='Deborah', ssn='418-78-9128', occupation='Manufacturing systems engineer', age=47),
 Row(last_name='Jones', first_name='Mary', ssn='319-27-2359', occupation='Medical illustrator', age=47),
 Row(last_name='Hughes', first_name='Paul', ssn='659-82-7416', occupation='Environmental education officer', age=47)]

In [43]:
display(dataDF.orderBy('age').take(5))

[Row(last_name='Clark', first_name='Aaron', ssn='119-31-2898', occupation='Water engineer', age=1),
 Row(last_name='Watson', first_name='Carrie', ssn='839-77-6452', occupation='Insurance account manager', age=1),
 Row(last_name='Hunter', first_name='Jeffrey', ssn='474-66-8191', occupation='Ranger/warden', age=1),
 Row(last_name='Lewis', first_name='Tyrone', ssn='654-41-7869', occupation='Teacher, music', age=1),
 Row(last_name='Green', first_name='Jeff', ssn='539-24-1751', occupation='Financial adviser', age=1)]

In [44]:
print(dataDF.count())
print(dataDF.distinct().count())

10000
10000


In [45]:
tempDF = sqlContext.createDataFrame([("Joe", 1), ("Joe", 1), ("Anna", 15), ("Anna", 12), ("Ravi", 5)], ('name', 'score'))

In [46]:
tempDF.show()

+----+-----+
|name|score|
+----+-----+
| Joe|    1|
| Joe|    1|
|Anna|   15|
|Anna|   12|
|Ravi|    5|
+----+-----+



In [47]:
tempDF.distinct().show()

+----+-----+
|name|score|
+----+-----+
| Joe|    1|
|Ravi|    5|
|Anna|   12|
|Anna|   15|
+----+-----+



In [48]:
dataDF.groupBy('occupation').count().show(truncate=False)

+---------------------------------------+-----+
|occupation                             |count|
+---------------------------------------+-----+
|Catering manager                       |11   |
|Designer, ceramics/pottery             |18   |
|Diplomatic Services operational officer|18   |
|Retail merchandiser                    |15   |
|Librarian, academic                    |18   |
|Engineer, aeronautical                 |10   |
|Early years teacher                    |22   |
|Patent examiner                        |16   |
|Occupational hygienist                 |13   |
|English as a second language teacher   |7    |
|Primary school teacher                 |6    |
|Estate agent                           |19   |
|Clinical molecular geneticist          |12   |
|Control and instrumentation engineer   |13   |
|Transport planner                      |15   |
|Applications developer                 |10   |
|Art therapist                          |16   |
|Investment banker, operational         

In [49]:
dataDF.groupBy().avg('age').show(truncate=False)

+--------+
|avg(age)|
+--------+
|23.6432 |
+--------+



In [51]:
sampledDF = dataDF.sample(withReplacement=False, fraction=0.10)
print(sampledDF.count())
sampledDF.show()

1000
+---------+----------+-----------+--------------------+---+
|last_name|first_name|        ssn|          occupation|age|
+---------+----------+-----------+--------------------+---+
|    Simon|     Logan|062-37-6157|Health service ma...| 32|
|Patterson|     Bobby|063-74-9852|International aid...|  1|
|  Parsons|   Michael|304-93-2898|  Surveyor, quantity| 38|
|    Oneal|     Linda|766-04-5107|Commercial/reside...| 45|
|   Garcia|  Danielle|717-05-1050|Estate manager/la...|  2|
| Valencia| Christina|293-28-6195|Sport and exercis...| 44|
|    Clark|     Megan|592-99-2891|Waste management ...|  6|
|Schroeder|     David|046-68-6425| Early years teacher| 42|
|    Scott|   Michael|534-04-9753|Chartered accountant| 29|
|    Allen|   Rebecca|123-75-7691|Further education...| 17|
|    Perez|     Karen|142-89-5887|Environmental man...|  2|
|     Cole|   Cynthia|611-12-4910|Community arts wo...| 42|
|  Watkins|     Isaac|795-31-5815|Geographical info...|  2|
| Anderson|     Glenn|809-59-4131| 

In [52]:
print(dataDF.sample(withReplacement=False, fraction=0.05).count())

512
